In [ ]:
%load_ext autoreload
%reload_ext autoreload

%autoreload 2

import numpy as np
import pandas as pd
from scipy import stats
import datetime
import os
import re
import logging

logger = logging.getLogger(__name__)
import pandas as pd
import numpy as np
from kedro.framework.context import load_context
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 300)
"""Visualize"""
import cufflinks as cf

import plotly
import plotly.offline as py
import plotly.graph_objs as go

cf.go_offline() # required to use plotly offline (no account required).
py.init_notebook_mode() # graphs charts inline (IPython).
"""Sklearn"""
from sklearn.metrics.pairwise import cosine_similarity
import talib
context = load_context('/Users/Zishuo_Li/projects/pricing_airbnb/', env='base')
catalog = context.catalog

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

from prc.nodes.ds import model

In [410]:
df = catalog.load("fea_output")

2020-06-01 13:23:41,959 - kedro.io.data_catalog - INFO - Loading data from `fea_output` (CSVDataSet)...


In [413]:
embedding_cols = ["neighbourhood_group", "room_type", "month"]
val_cols = ["price", "minimum_nights", "lag", "number_of_reviews", "calculated_host_listings_count"]

num_dimensions = len(val_cols)
num_samples = 1

In [414]:
df = df.dropna(subset=embedding_cols + val_cols, how="any")
df = df.set_index(["listing_id", "date"])

In [415]:
df = df[val_cols + embedding_cols + ["available"]]
df.head()

price  minimum_nights       lag  number_of_reviews  \
listing_id date                                                                
26520      2020-05-08  4.584967        0.693147  1.098612                 16   
           2020-05-09  4.584967        0.693147  1.386294                 16   
           2020-05-10  4.584967        0.693147  1.609438                 16   
           2020-05-11  4.584967        0.693147  1.791759                 16   
           2020-05-12  4.584967        0.693147  1.945910                 16   

                       calculated_host_listings_count neighbourhood_group  \
listing_id date                                                             
26520      2020-05-08                               6            Brooklyn   
           2020-05-09                               6            Brooklyn   
           2020-05-10                               6            Brooklyn   
           2020-05-11                               6            Brooklyn   
           2020-05-12                               6            Brooklyn   

                          room_type  month available  
listing_id date                                       
26520      2020-05-08  Private room      5         t  
           2020-05-09  Private room      5         t  
           2020-05-10  Private room      5         t  
           2020-05-11  Private room      5         t  
           2020-05-12  Private room      5         t

In [433]:
df_true = df[df["available"] == "t"]
df_false = df[df["available"] != "t"]

In [434]:
df_sampled = pd.concat([df_false.sample(len(df_true)), df_true], axis=0)

In [435]:
df_sampled.shape

(1376604, 9)

In [438]:
x_val = df_sampled[val_cols].values

y = (df_sampled["available"] == "t").astype(int)

In [447]:
embed_lookup, x_embed_raw = build_embedding_dic(df_sampled, embedding_cols)

In [457]:
    neighbourhood_group_input = tf.keras.layers.Input(shape=1)
    neighbourhood_group_embed_layer = tf.keras.layers.Embedding(
        input_dim=len(embed_lookup["neighbourhood_group"]), output_dim=5
    )(neighbourhood_group_input)

In [462]:
tf.keras.layers.Flatten()(neighbourhood_group_embed_layer)

<tf.Tensor 'flatten_1/Identity:0' shape=(None, 5) dtype=float32>

In [463]:
def build_model(num_dimensions, num_samples, embed_lookup, lr=0.001):
    """Based model."""
    input_layer = tf.keras.layers.Input(shape=num_dimensions)
    # add embedding layer for neighbourhood
    neighbourhood_group_input = tf.keras.layers.Input(shape=1)
    neighbourhood_group_embed_layer = tf.keras.layers.Embedding(
        input_dim=len(embed_lookup["neighbourhood_group"]), output_dim=5
    )(neighbourhood_group_input)
    neighbourhood_flatten = tf.keras.layers.Flatten()(neighbourhood_group_embed_layer)
    
    # add embedding layer for room
    room_type_input = tf.keras.layers.Input(shape=1)
    room_type_embed_layer = tf.keras.layers.Embedding(
        input_dim=len(embed_lookup["room_type"]), output_dim=5
    )(room_type_input)
    room_type_flatten = tf.keras.layers.Flatten()(room_type_embed_layer)

    # add embedding layer for month
    month_input = tf.keras.layers.Input(shape=1)
    month_embed_layer = tf.keras.layers.Embedding(
        input_dim=len(embed_lookup["month"]), output_dim=12
    )(month_input)
    month_flattern = tf.keras.layers.Flatten()(month_embed_layer)
    
    concat_layer = tf.keras.layers.concatenate(
        [
            input_layer,
            neighbourhood_flatten,
            room_type_flatten,
            month_flattern,
        ]
    )
    kl_divergence_function = lambda q, p, _: tfd.kl_divergence(q, p) / tf.cast(
        num_samples, dtype=tf.float32
    )
    dense_layer = tfp.layers.DenseFlipout(
        units=1,
        activation="sigmoid",
        kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
        bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
        kernel_divergence_fn=kl_divergence_function,
    )(concat_layer)

    model = tf.keras.Model(
        inputs=[input_layer, neighbourhood_group_input, room_type_input, month_input],
        outputs=dense_layer,
    )

    optimizer = tf.keras.optimizers.Adam(lr=lr)
    model.compile(optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model


In [464]:
model = build_model(num_dimensions, num_samples, embed_lookup, lr=0.001)

In [465]:
model.summary()

Model: "model_101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_162 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_163 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_164 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_59 (Embedding)        (None, 1, 5)         25          input_162[0][0]                  
__________________________________________________________________________________________

In [466]:
model.fit([x_val, x_embed_raw[0], x_embed_raw[1], x_embed_raw[2]], y, epochs=10)

Epoch 1/10
43019/43019 [==============================] - 53s 1ms/step - loss: 7.4050 - accuracy: 0.5668 0s - loss:
Epoch 2/10
43019/43019 [==============================] - 41s 954us/step - loss: 5.2265 - accuracy: 0.5680
Epoch 3/10
43019/43019 [==============================] - 48s 1ms/step - loss: 5.2162 - accuracy: 0.5677
Epoch 4/10
43019/43019 [==============================] - 51s 1ms/step - loss: 5.2207 - accuracy: 0.5689
Epoch 5/10
43019/43019 [==============================] - 61s 1ms/step - loss: 5.2208 - accuracy: 0.5672
Epoch 6/10
43019/43019 [==============================] - 47s 1ms/step - loss: 5.2278 - accuracy: 0.5688
Epoch 7/10
43019/43019 [==============================] - 51s 1ms/step - loss: 5.2279 - accuracy: 0.5687
Epoch 8/10
43019/43019 [==============================] - 55s 1ms/step - loss: 5.2299 - accuracy: 0.5682
Epoch 9/10
43019/43019 [==============================] - 50s 1ms/step - loss: 5.2429 - accuracy: 0.5679
Epoch 10/10
43019/43019 [=================

In [150]:
last_layer = model.layers[-1]

# without embedding

In [359]:
def build_model(num_dimensions, num_samples, lr=0.001):
    kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) /  tf.cast(num_samples, dtype=tf.float32))

    input_layer = tf.keras.layers.Input(shape=num_dimensions)
    dense_layer = tfp.layers.DenseFlipout(
        units=1,
        activation='sigmoid',
        kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
        bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
        kernel_divergence_fn=kl_divergence_function)(input_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=dense_layer)

    optimizer = tf.keras.optimizers.Adam(lr=lr)
    model.compile(optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [360]:
x_val.shape

(1484260, 5)

In [381]:
num_samples = 600
model = build_model(num_dimensions, num_samples, lr=0.0001)


model.fit(x_val, y, epochs=2, batch_size=num_samples)

Epoch 1/2
1/1 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.8000
Epoch 2/2
1/1 [==============================] - 0s 945us/step - loss: nan - accuracy: 0.5000


In [380]:
val_cols

['price',
 'minimum_nights',
 'lag',
 'number_of_reviews',
 'calculated_host_listings_count']

# EVAL

In [10]:
candidate_w_bs = [(model.layers[-1].kernel_posterior.sample().numpy(), model.layers[-1].bias_posterior.sample().numpy())]

In [68]:
y_pred = model.predict(x)

In [69]:
df["y_pred"] = y_pred

In [70]:
df

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,lag,month,y_pred
0,2539,2019-05-04,t,5.003946,$149.00,0.0,730.0,0.000000,1.609438,NaN
1,2539,2019-05-05,t,4.859812,$129.00,0.0,730.0,0.693147,1.609438,NaN
2,2539,2019-05-06,t,4.859812,$129.00,0.0,730.0,1.098612,1.609438,NaN
3,2539,2019-05-07,t,4.859812,$129.00,0.0,730.0,1.386294,1.609438,NaN
4,2539,2019-05-08,t,4.859812,$129.00,0.0,730.0,1.609438,1.609438,NaN
...,...,...,...,...,...,...,...,...,...,...
2949620,34369498,2019-06-28,t,5.768321,$320.00,0.0,1125.0,4.025352,1.791759,NaN
2949621,34369498,2019-06-29,t,5.768321,$320.00,0.0,1125.0,4.043051,1.791759,NaN
2949622,34369498,2019-06-30,t,5.768321,$320.00,0.0,1125.0,4.060443,1.791759,NaN
2949623,34369498,2019-07-01,t,5.768321,$320.00,0.0,1125.0,4.077537,1.945910,NaN
